# convert to YOLO format

In [6]:
import torch
from ultralytics import YOLO
import cv2
from IPython.display import display, Image
import matplotlib.pyplot as plt
import pytesseract
from PIL import Image
import os
import easyocr
import numpy as np

**To run the code, first run the next cell, then run the odometer_reader method (cell after that), and then after setting the path to the test images folder and weights, run the last cell.**


Methods of this project:
* crop_odometer_with_highest_confidence
   - *To crop the odometer with the highest condidence*
* reduce_noise
   - *Using morphology to reduce noise*
* preprocess_image
   - *preprocessing the cropped image (gray, binary, reduce noise)*
* extract_odometer_reading
   - *Runs the OCR on the cropped image to extract the numeric results*
* process_images
   - *reads the images and call gives the results*


In [8]:
def crop_odometer_with_highest_confidence(image, results):
    max_confidence = -1
    best_bbox = None
    for result in results:
        for box in result.boxes:
            confidence = box.conf
            if confidence > max_confidence:
                max_confidence = confidence
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cropped_image = image[y1:y2, x1:x2]
                best_bbox = (cropped_image, (x1, y1, x2, y2))
    return best_bbox
def reduce_noise(image):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # kernel = np.ones((10,10), np.uint8)
    # opening = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
    # Closing (dilation followed by erosion)
    # closing = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    denoised = reduce_noise(thresh)
    # plt.imshow(denoised)
    # plt.show()
    return denoised

def extract_odometer_reading(cropped_image, reader):
    preprocessed_image = preprocess_image(cropped_image)
    
    # Run OCR
    results = reader.readtext(preprocessed_image, min_size=2)
    
    # We only need numeric output, so we filter the output to get the numeric results only
    numeric_results = []
    for res in results:
        text = res[1]
        numeric_text = ''.join(filter(str.isdigit, text))
        if numeric_text:
            numeric_results.append(numeric_text)
    
    return numeric_results

def process_images(image_dir, model, reader):
    
    image_files = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if (img.endswith('.jpeg') or img.endswith('.jpg'))]
    
    for image_path in image_files:
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    
        if image is None:
            print(f"Failed to load image: {image_path}")
            continue
        
        results = model(image, save=False)
        
        # Convert image from BGR to RGB for displaying
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # choose the odometer with the highest condidence (CUZ there might be multiple boxes in one image and we only need one)
        best_crop_bbox = crop_odometer_with_highest_confidence(image, results)
        if best_crop_bbox:
            cropped_image, bbox = best_crop_bbox
            odometer_readings = extract_odometer_reading(cropped_image, reader)
            
            for reading in odometer_readings:
                if (reading == ""):
                    print(f'Odometer reading for {image_path}: can"t read')
                else:
                    print(f'Odometer reading for {image_path}: {reading}')
        
        # Display the image with bounding box
        x1, y1, x2, y2 = bbox
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
        plt.imshow(image_rgb)
        plt.show()

In [7]:
def odometer_reader(image_dir, model_path):
    model = YOLO(model_path)
    reader = reader = easyocr.Reader(['en'])
    process_images(image_dir, model, reader)

In [ ]:
img_dir = "imagefolderpath" # path to the test folder (which contains images from dashboards)
model_path = "weights\last.pt" # path to the weights of the model
odometer_reader(img_dir,model_path)